# Linear Regression Toolkit - Demo Notebook

This notebook demonstrates the Linear Regression toolkit with three experiments:
1. **Straight Line with Noise** - Basic linear regression
2. **Collinearity and Ridge Regularization** - Effect of L2 regularization
3. **Polynomial Regression** - Comparing different polynomial degrees

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from linear_models import LinearRegressionClosedForm
from metrics import mse, r2_score
from selection import train_test_split
from plotting import plot_predictions, plot_residuals, plot_fitted_curve

np.random.seed(42)
print("Imports successful!")

---
# Experiment 1: Straight Line with Noise

**Goal:** Demonstrate basic linear regression on data generated from y = 3 + 2x + ε

## Generate Synthetic Data

In [ ]:
print("=" * 60)
print("EXPERIMENT 1: Straight Line with Noise")
print("=" * 60)

# Generate synthetic data: y = 3 + 2x + ε
n_samples = 200
X = np.random.uniform(0, 10, n_samples).reshape(-1, 1)
epsilon = np.random.randn(n_samples) * 1.0  # Noise with std=1
y = 3 + 2 * X.flatten() + epsilon

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

## Fit Model

In [ ]:
model = LinearRegressionClosedForm(fit_intercept=True, alpha=0.0)
model.fit(X_train, y_train)

print(f"\nTrue parameters: intercept=3.0, slope=2.0")
print(f"Learned parameters: intercept={model.intercept_:.4f}, slope={model.coef_[0]:.4f}")

## Evaluate Performance

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mse(y_train, y_train_pred)
test_mse = mse(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\nTraining MSE: {train_mse:.4f}")
print(f"Test MSE: {test_mse:.4f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

## Visualize Results

In [ ]:
plot_predictions(y_test, y_test_pred, title="Experiment 1: Predictions vs True Values")

In [ ]:
plot_fitted_curve(X_test, y_test, y_test_pred, title="Experiment 1: Fitted Line")

---
# Experiment 2: Collinearity and Ridge Regularization

**Goal:** Demonstrate how Ridge regularization affects coefficient magnitude when features are highly correlated

## Generate Collinear Data

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 2: Collinearity and Ridge Regularization")
print("=" * 60)

n_samples = 100
x1 = np.random.randn(n_samples)
x2 = x1 + 0.01 * np.random.randn(n_samples)  # Highly correlated with x1
X = np.column_stack([x1, x2])
y = 3 * x1 + 2 * x2 + np.random.randn(n_samples) * 0.5

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nFeature correlation: {np.corrcoef(x1, x2)[0, 1]:.4f}")
print(f"(Close to 1.0 indicates high collinearity)")

## Test Different Alpha Values

In [ ]:
alphas = [0.0, 1e-3, 1e-2, 1e-1, 1.0]
results = []

print("\n" + "-" * 80)
print(f"{'Alpha':<10} {'Test MSE':<12} {'||coef||₂':<12} {'Coefficients'}")
print("-" * 80)

for alpha in alphas:
    model = LinearRegressionClosedForm(alpha=alpha)
    model.fit(X_train, y_train)
    
    y_test_pred = model.predict(X_test)
    test_mse_val = mse(y_test, y_test_pred)
    coef_norm = np.linalg.norm(model.coef_)
    
    results.append({
        'alpha': alpha,
        'test_mse': test_mse_val,
        'coef_norm': coef_norm,
        'coef_': model.coef_.copy()
    })
    
    print(f"{alpha:<10.4f} {test_mse_val:<12.4f} {coef_norm:<12.4f} {model.coef_}")

print("-" * 80)

## Visualize Impact of Regularization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot MSE vs alpha
axes[0].plot([r['alpha'] for r in results], [r['test_mse'] for r in results], 'o-', markersize=8)
axes[0].set_xlabel('Alpha (Regularization Strength)', fontsize=12)
axes[0].set_ylabel('Test MSE', fontsize=12)
axes[0].set_title('Test MSE vs Regularization', fontsize=14)
axes[0].set_xscale('symlog', linthresh=1e-4)
axes[0].grid(True, alpha=0.3)

# Plot coefficient norm vs alpha
axes[1].plot([r['alpha'] for r in results[1:]], [r['coef_norm'] for r in results[1:]], 'o-', markersize=8, color='red')
axes[1].set_xlabel('Alpha (Regularization Strength)', fontsize=12)
axes[1].set_ylabel('||coef||₂', fontsize=12)
axes[1].set_title('Coefficient Norm vs Regularization', fontsize=14)
axes[1].set_xscale('log')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Discussion - Experiment 2

### Observations:

As alpha (regularization strength) increases:

1. **Coefficient Magnitude Decreases**: The L2 norm of the coefficient vector shrinks consistently. Ridge regression penalizes large coefficients, forcing them toward zero but never exactly zero.

2. **Bias-Variance Tradeoff**: With collinear features, OLS (alpha=0) produces unstable coefficient estimates with high variance. Ridge regression introduces bias but reduces variance, often improving generalization.

3. **Test MSE Pattern**: Initially, small amounts of regularization may reduce test MSE by preventing overfitting. However, excessive regularization (large alpha) increases bias too much, degrading performance.

4. **Practical Insight**: When features are highly correlated, Ridge regression distributes weights more evenly between them rather than arbitrarily assigning all weight to one feature.

**Key Takeaway**: Ridge regularization is particularly effective when dealing with multicollinearity in your features.

---
# Experiment 3: Polynomial Regression

**Goal:** Compare linear regression with different polynomial degrees to fit nonlinear data

## Generate Nonlinear Data

In [ ]:
print("\n" + "=" * 60)
print("EXPERIMENT 3: Polynomial Regression")
print("=" * 60)

n_samples = 150
X_raw = np.random.uniform(-3, 3, n_samples)
epsilon = np.random.randn(n_samples) * 0.5
y = 1 + 2 * X_raw - 0.3 * X_raw**2 + epsilon

# Sort for plotting
sort_idx = np.argsort(X_raw)
X_raw = X_raw[sort_idx]
y = y[sort_idx]

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw.reshape(-1, 1), y, test_size=0.2, random_state=42
)

print(f"\nTrue function: y = 1 + 2x - 0.3x²")
print(f"Training samples: {len(X_train_raw)}")
print(f"Test samples: {len(X_test_raw)}")

## Helper Function for Polynomial Features

In [ ]:
def polynomial_features(X, degree):
    """
    Create polynomial features up to specified degree.
    
    Args:
        X (np.ndarray): Input features, shape (n_samples, 1)
        degree (int): Maximum polynomial degree
    
    Returns:
        np.ndarray: Polynomial features, shape (n_samples, degree)
    """
    X = X.flatten()
    features = np.column_stack([X**d for d in range(1, degree + 1)])
    return features

## Compare Different Polynomial Degrees

In [ ]:
degrees = [1, 2, 5]
models = {}

print("\n" + "-" * 60)

for deg in degrees:
    print(f"\n--- Degree {deg} ---")
    
    # Create polynomial features
    X_train_poly = polynomial_features(X_train_raw, deg)
    X_test_poly = polynomial_features(X_test_raw, deg)
    
    # Fit model
    model = LinearRegressionClosedForm(alpha=0.0)
    model.fit(X_train_poly, y_train)
    models[deg] = model
    
    # Evaluate
    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)
    
    train_mse_val = mse(y_train, y_train_pred)
    test_mse_val = mse(y_test, y_test_pred)
    train_r2_val = r2_score(y_train, y_train_pred)
    test_r2_val = r2_score(y_test, y_test_pred)
    
    print(f"Training MSE: {train_mse_val:.4f}, R²: {train_r2_val:.4f}")
    print(f"Test MSE: {test_mse_val:.4f}, R²: {test_r2_val:.4f}")

print("\n" + "-" * 60)

## Visualize Fitted Curves

In [ ]:
X_plot = np.linspace(-3, 3, 300).reshape(-1, 1)

plt.figure(figsize=(12, 6))
plt.scatter(X_raw, y, alpha=0.5, s=20, label='True Data', edgecolors='k')

colors = ['green', 'blue', 'red']
for deg, color in zip(degrees, colors):
    X_plot_poly = polynomial_features(X_plot, deg)
    y_plot_pred = models[deg].predict(X_plot_poly)
    plt.plot(X_plot, y_plot_pred, color=color, lw=2, label=f'Degree {deg}')

# Plot true function
y_true_plot = 1 + 2 * X_plot.flatten() - 0.3 * X_plot.flatten()**2
plt.plot(X_plot, y_true_plot, 'k--', lw=2, alpha=0.7, label='True Function')

plt.xlabel('X', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Polynomial Regression: Comparing Degrees', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Discussion - Experiment 3

### Which degree fits best and why?

**Degree 1 (Linear)**: Underfits the data because the true relationship is quadratic. The linear model cannot capture the curvature, resulting in higher MSE and lower R².

**Degree 2 (Quadratic)**: **Best fit!** This matches the true data-generating process (y = 1 + 2x - 0.3x²). It captures the curvature accurately without overfitting, achieving the lowest test MSE and highest R².

**Degree 5 (Quintic)**: Overfits the training data. While it achieves low training MSE, it captures noise rather than the underlying pattern. The model shows wiggly behavior outside the training range and has higher test MSE than degree 2.

### Key Insight

Model complexity should match the complexity of the underlying process:
- **Too simple** (degree 1) misses important patterns
- **Too complex** (degree 5) fits noise instead of signal
- **Just right** (degree 2) captures the true relationship

Cross-validation or holdout validation (as done here) helps identify the right complexity level.

**Practical Takeaway**: Start with simpler models and increase complexity only when justified by improved test performance.

---
# Summary

This notebook demonstrated three important aspects of linear regression:

1. **Basic Linear Regression**: Successfully recovered true parameters from noisy data
2. **Ridge Regularization**: Showed how L2 penalty helps with collinear features
3. **Polynomial Features**: Illustrated the bias-variance tradeoff and model complexity

All experiments used our custom `LinearRegressionClosedForm` implementation with the closed-form normal equation solution!